In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

前の章ではデータに正解ラベルがついていた．これを**教師あり学習** という．  
ここでは，ラベルがない場合，**教師なし学習**を考えてみる．  
<br>
教師なし学習では，データだけからあるパターンを見つける．  
例えば，質問サイトで，ある質問に関連するページを見つけるには，文書ごとの類似度を全て算出すれば良いと考えられる．  
しかしこの方法は計算量が膨大になってしまう．  
素早く関連する文書を見つけるためには，**クラスタリング**という手法が使えそうである．  
こちらもテキストデータ同士で類似度を算出する方法が必要になるが，ここではSciKitライブラリの機能を使ってみたいと思う．

# 文書の関連性を計測する
まずテキストデータを意味のある数字に変換したい．

## やってはいけないこと
テキストデータの類似度を求めるには，**レーベンシュタイン距離(編集距離とも呼ばれる)** が利用できそう．  
文字ごとの編集距離を求めると時間がかかり過ぎてしまうので，単語を最小単位として扱い，文章全体で編集距離を計測しようとも思ってみる．  
しかし依然編集距離のオーダーは大きく，時間がかかる．  
また，同じ単語が2つの文書に出現していて，位置が異なる場合にも編集距離は発生してしまうので，この方法はロバストであるとは言えない．

## どうやるべきか
**bag-of-words**を使ってみる．  
単語の出現回数を特徴量として扱うことで，1つの文書を1つのベクトルとして扱える．  
(しかし，この方法で最近傍点を見つけるのには時間がかかりすぎる．)  
**bag-of-words**によってクラスタリング処理を行う手順を示す．  
1. 各文書から特徴量を抽出し，特徴ベクトルの形で保存する
1. 特徴ベクトルに対してクラスタリングを行う
1. 投稿された質問文書に対して，クラスタを決定する
1. このクラスタに属する文書を他に集め，多様性を増す

# 前処理: 共通する単語の出現回数を類似度として計測する
scikitのCountVectorizerでbag-of-wordsを作れる  

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
print(vectorizer)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


min_dfはその数より出現回数の小さい単語を無視する．  
analizerは，単語レベルで出現回数のカウントを行なっていることを意味する．  
token_pattenでは単語の決定方法を定義．例えばcross-validatedをcrossとvalidatedに分けるかを設定できる  

In [3]:
content = ["How to format my hard disk", " Hard disk format problems "]
X = vectorizer.fit_transform(content)
print("vocabulary:", vectorizer.get_feature_names())
print(X.toarray(), "\n\n", X.toarray().transpose())

vocabulary: ['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']
[[1 1 1 1 1 0 1]
 [1 1 1 0 0 1 0]] 

 [[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


## 単語を数える

In [4]:
txt1 = "This is a toy post about machine learning. Actually, it contains not much interesting stuff."
txt2 = "Imaging databases provide storage capabilities."
txt3 = "Most imaging databases safe images permanently."
txt4 = "Imaging databases store data."
txt5 = "Imaging databases store data. Imaging databases store data. Imaging databases store data."

posts = [txt1, txt2, txt3, txt4, txt5]
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print(f"#samples: {num_samples}, #features: {num_features}")
print(vectorizer.get_feature_names())

# 新しい文書のベクトル化
new_post = ["imaging databases"] # Iterable over raw text documents expected, string object received.対策
new_post_vec = vectorizer.transform(new_post)
print(new_post_vec) # 疎なベクトルなので，0でない値のみが表示される
print(new_post_vec.toarray()) # ベクトル全体を表示

#samples: 5, #features: 25
['about', 'actually', 'capabilities', 'contains', 'data', 'databases', 'images', 'imaging', 'interesting', 'is', 'it', 'learning', 'machine', 'most', 'much', 'not', 'permanently', 'post', 'provide', 'safe', 'storage', 'store', 'stuff', 'this', 'toy']
  (0, 5)	1
  (0, 7)	1
[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


2つの文書のユークリッド距離を計算してみる

In [5]:
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [6]:
import sys

def distances(posts, new_post, distance):
    best_doc = None
    best_dist = sys.maxsize
    best_i = None
    new_post_vec = vectorizer.transform(new_post)
    for i in range(num_samples):
        post = posts[i]
        if post == new_post: continue
        post_vec = X_train.getrow(i)
        d = distance(post_vec, new_post_vec)
        print("=== Post %i with dist=%.2f: %s" % (i, d, post))
        if d < best_dist:
            best_dist = d
            best_i = i

    print("Best post is %i with dist %.2f" % (best_i, best_dist))

distances(posts, new_post, dist_raw)

=== Post 0 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.73: Imaging databases provide storage capabilities.
=== Post 2 with dist=2.00: Most imaging databases safe images permanently.
=== Post 3 with dist=1.41: Imaging databases store data.
=== Post 4 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist 1.41


## 単語の出現回数ベクトルを正規化する
Post3とPost4の結果は同じにならないと不自然なので，正規化する

In [7]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = v1_normalized - v2_normalized
    return sp.linalg.norm(delta.toarray())

distances(posts, new_post, dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.92: Most imaging databases safe images permanently.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist 0.77


## 重要度の低い単語を取り除く
文書の分類に貢献しない単語，**ストップワード** を取り除く

In [8]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
print(sorted(vectorizer.get_stop_words())[:20])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst']


In [9]:
X_train = vectorizer.fit_transform(posts)
distances(posts, new_post, dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases safe images permanently.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist 0.77


## ステミング
語形変化に対応し，語幹(stem)を単語として扱いたい．  
scikitにはその機能がないため，**Natural Language Toolkit(NLTK)** が必要
```
$ pip install nltk
```

In [10]:
import nltk.stem
stemmer = nltk.stem.SnowballStemmer('english') # 他にも色々ステマーはあるらしい

words = ["graphics", "imaging", "image", "imagination", "imagine", "buy", "buying", "bought"]
stemmed = [stemmer.stem(w) for w in words]
for w, s in zip(words, stemmed): print(w, "->", s)

graphics -> graphic
imaging -> imag
image -> imag
imagination -> imagin
imagine -> imagin
buy -> buy
buying -> buy
bought -> bought


ステミングを行い，トークン化と正規化を行うようCountVectorizerのbuild_analizerメソッドをオーバーライド

In [15]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer() # 文書を小文字に変換 + 単語の抜き出し
        return lambda doc: (stemmer.stem(w) for w in analyzer(doc)) # ステミング

vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')
X_train = vectorizer.fit_transform(posts)
distances(posts, new_post, dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.63: Most imaging databases safe images permanently.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist 0.63


## TF-IDFを用いる
頻出単語はmax_dfを0.9に設定するなどすることで除外できるが，0.89は除外できない．  
この閾値を考えているとキリがない．  
また，これでは除外されなかった頻出単語とそうでない単語で識別性能(クラスタリングへの貢献度)が変わってきてしまう．  
そこで，文書内での単語の重要度として，単語の出現数の代わりに**TF-IDF(term frequency - inverse document frequency)** を使ってベクトル化する．  
<br>
tf: Term Frequency, 与えられた文書の中での単語の出現頻度
$$ tf = \frac{文書docにおける単語termの出現頻度}{文書docにおける全単語の出現頻度の和(docの単語数?)} = 文書docにおける単語termの割合$$
<br>
idf: Inverse Document Frequency, 逆文書頻度,単語が「レア」なら高い値を，「他の文書にも良く出現する単語」なら低い値を示すもの．  
レアな単語はその文書の特徴を判別するのに有用であるとする．  
$$ idf = \log \Bigl( \frac{全文書数}{単語termを含む文書数} \Bigr) = - \log \Bigl( \frac{単語termを含む文書数}{全文書数} \Bigr) $$
これは単語termを含むという事象に対する情報量，エントロピーの式になっている．  
この文書が単語termを含む珍しさ，含むことの予測しにくさを表している．  
そして，TF-IDFは
$$ tfidf = tf \times idf $$
で表される．  
<br>
TF-IDFによって，「その単語がその文書でよく出現するほど」，「その単語が他の文書と比べてレアなほど」大きな値になる．  
つまり，TF-IDFはその単語が文書全体の中でどれだけ特徴的かを表す指標である．  
その文書内にその単語が出現しなければ,TF-IDFは0になる．  
参考: https://dev.classmethod.jp/machine-learning/yoshim_2017ad_tfidf_1-2/

In [12]:
import math
def tfidf(term, doc, docset):
    tf = float(doc.count(term)) / sum(doc.count(w) for w in set(doc))
    idf = math.log(float(len(docset)) / (len([doc for doc in docset if term in doc])))
    return tf * idf

In [13]:
doc1, doc2, doc3 = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [doc1, doc2, doc3]
print(tfidf("a", doc1, D))
print(tfidf("b", doc2, D))
print(tfidf("a", doc3, D))
print(tfidf("b", doc3, D)) # doc2におけるbはdoc3におけるbの2倍重要(2回出てきてるから)
print(tfidf("c", doc3, D))

0.0
0.27031007207210955
0.0
0.13515503603605478
0.3662040962227032


sklearnにはTF-IDFの値で単語をベクトル化してくれるTfidfVectorizer(CountVectorizerを継承したクラス)が用意されている．  
これをCountVectorizerの代わりに使う．  

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer() # 文書を小文字に変換 + 単語の抜き出し
        return lambda doc: (stemmer.stem(w) for w in analyzer(doc)) # ステミング

vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english')
X_train = vectorizer.fit_transform(posts)
distances(posts, new_post, dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.08: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases safe images permanently.
=== Post 3 with dist=0.92: Imaging databases store data.
=== Post 4 with dist=0.92: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist 0.86


## ここまでやってきたこと
1. 文書データをトークン化(bag-of-words化)する
1. 頻出しすぎる単語は取り除く (stop wordsの除外)
1. 滅多に使われない単語は，新しい文書でも使われる可能性が低いため，取り除く(min_dfの設定)
1. 残った単語について，その出現回数をカウントする
1. ↑のカウント手法だと単語ごとにクラスタリングへの貢献度が変わってきてしまうので，TF-IDFを文書の特徴ベクトルの要素とする．

このbag-of-wordsによるアプローチは，単純でありながら，優れた性能を持つ．  
しかし，以下のような欠点を持つ．  
- 単語の前後関係について考慮していない．たとえば，Car hits wall と Wall hits carが同じものとして扱われる．
- 否定的な意味を捉えられない． たとえば．I will eat ice creamと I will not eat ice cream の特徴ベクトルが非常に似ることになる．  
    - この問題は，単語を個別に考えるのではなく，ペア(バイグラム)や3つの連続する単語(トリグラム)を一つのまとまりとしてカウントすることで解決できる．  
- タイプミスに対応できない． databaseとdatabasが別の単語として扱われる．

# クラスタリング
ここでようやく文書の内容を適切に表現しているであろう特徴ベクトルを手にすることができた．  
クラスタリング手法の多くは，次の2つの手法のどちらかに該当する．  
- フラットクラスタリング(flat clustering)
    - クラスタ間の関係を考慮しない
    - 全てのデータがどこか1つのクラスタに属するよう分割
    - 前もってクラスタの数を決める必要があるアルゴリズムが多い
- 階層的クラスタリング(hierarchical clustering)
     - 普通にフラットクラスタリングを行なった後，さらに近いクラスタ同士を一つのクラスタにまとめる親クラスタを作ることを再帰的に繰り返す
     - 最終的には全てのデータが一つの大きなクラスタに属すことになる
     - 前もってクラスタの数を決める必要はない． 決めることはできるが，処理効率を落とすことになるらしい
 
Scikit-learnでできる様々なクラスタリング手法の説明  
https://scikit-learn.org/dev/modules/clustering.html

## KMeans
もっとも広く用いられるフラットクラスタリング手法
1. クラスタの数num_clustersを指定して初期化
1. num_clustersの数だけランダムにデータを選び出し，それらの特徴ベクトルを各クラスタの中心点とする
1. その他のデータは，もっとも近い中心点を持つクラスタに所属させる
1. 全てのデータのクラスタが決まったら，各クラスタの中心を新たに計算し，そこに中心点を移動させる
1. 中心点の移動量が閾値を下回る(収束)，または指定の繰り返し回数に達するまで2~4を繰り返す

ch03/plot_kmeans_example.pyを参照，実行
![01](ch03/1400_03_01.png)
![02](ch03/1400_03_02.png)
![03](ch03/1400_03_03.png)
![04](ch03/1400_03_04.png)